# Understanding Optimisers - Example Implementation

Optimization plays a crucial role in decision-making processes across various industries and disciplines. Whether it's allocating resources efficiently, scheduling tasks, or selecting projects to maximize benefits under constraints, optimization techniques provide systematic and quantitative methods to arrive at the best possible solutions.

In this notebook, we embark on a practical journey to understand how optimization can be applied to a real-world scenario using Python. Specifically, we will:

- **Simulate a Business Automation Scenario:** We'll create a set of business-as-usual (BAU) process automation projects, each with associated benefits such as hours saved per month, costs like hours required to implement, and specific skill requirements.
- **Utilize Advanced Data Structures:** We'll leverage Python's dataclasses to define clear and efficient data structures for our projects and team members, enhancing code readability and maintainability.
- **Implement an Optimization Model with `PuLP`:** We'll formulate and solve an optimization problem using PuLP, a powerful yet user-friendly linear programming library in Python. The goal is to select the optimal set of projects and assign team members effectively, maximizing the total hours saved while respecting various constraints.

## Objectives of the Notebook

  - Provide a step-by-step guide on how to model and solve an optimization problem in Python, suitable for learners new to optimization or Python programming.
  - Show how optimization techniques can be directly applied to manage resources and project selection in a business context.
  - Emphasize good coding practices by using modern Python features like dataclasses, and include comprehensive explanations through markdown cells.

## Scenario Overview

Imagine you are managing a team responsible for automating repetitive processes within an organization. You have:

- A list of potential automation projects, each requiring certain skills to implement and promising a specific amount of time saved per month once completed.
- A team of members with varying skills, Python proficiency levels (Basic, Intermediate, Advanced), and limited availability (hours they can allocate until the end of the financial year).
- Constraints such as only being able to run a maximum of two projects concurrently due to resource limitations.

Our Optimiser must:

- Select the best combination of projects to undertake that maximizes the total hours saved per month.
- Assign team members to projects in a way that respects their skill sets, availability, and the project's skill requirements.

## Key Concepts and Tools

### Optimization with PuLP

Our Optimiser is based on the commercial `Gurobi` engine - we're using `PuLP` as a FOSS stand-in for this code.

`PuLP` is an open-source linear programming (LP) library in Python that allows you to:

- Define decision variables, objective functions, and constraints in a mathematical optimization problem.
- Use built-in solvers like `CBC` (Coin-or branch and cut) to find optimal solutions.

By utilizing `PuLP`, we'll translate our business problem into a mathematical model, enabling us to compute the optimal project selection and team assignments programmatically.


### Data Classes (dataclasses)

Introduced in Python 3.7, dataclasses provide a decorator and functions for automatically adding special methods to user-defined classes. They simplify class creation by:

- Automatically generating methods like `__init__()` and `__repr__()`.
- Making the code cleaner and more focused on the actual data being stored.

We'll use dataclasses to define our `Project` and `TeamMember` classes, ensuring our code is concise and easy to understand.

### Constraints and Decision Variables

In our optimization model, we'll consider several constraints:

- Maximum Concurrent Projects: Limit the number of projects running at the same time.
- Skill Requirements: Ensure team members assigned to projects have the necessary skills and Python proficiency levels.
- Availability: Team members cannot be assigned to more hours than they have available.
- Assignment Constraints: Team members can only be assigned to selected projects.

Our decision variables will include:

- Project Selection Variables: Binary variables indicating whether a project is selected.
- Team Assignment Variables: Binary variables indicating whether a team member is assigned to a project.

## Structure of the Notebook

The notebook is organized into the following sections:

- **Environment Setup:** Installing and importing necessary libraries.
- **Data Definition:** Creating dataclasses for Project and TeamMember, and populating them with sample data.
- **Problem Formulation:** Defining the optimization problem's objective function and constraints using PuLP.
- **Model Implementation:** Translating the mathematical model into code.
 -**Solution and Interpretation:** Solving the optimization problem and interpreting the results.
- **Conclusion:** Summarizing what we've learned and potential next steps.

## Learning Outcomes

By the end of this notebook, you should be able to:

- Understand how to model an optimization problem in Python using PuLP.
- Utilize dataclasses to create clean and efficient data structures.
- Translate a real-world scenario into a mathematical optimization model.
- Interpret the results of the optimization and understand their implications in a business context.

## Prerequisites

To fully benefit from this notebook, you should have:

- Basic knowledge of Python programming.
- Familiarity with concepts in linear programming and optimization (helpful but not required, see "optimisation_maths_crash_course" for a refresher).
- An understanding of business resource allocation challenges.

## Let's Get Started!

In the following sections, we'll dive into the practical implementation, starting with setting up our environment and defining our data structures.

In [ ]:
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpStatus
from dataclasses import dataclass
from typing import List, get_type_hints
import pandas as pd

We'll create the data classes to hold our project and team member information. While Dataclasses can be a slightly round-about way of handling our data, it means that we have context associated with our information. And, if we scale up our project to hold many many more people and projects, we can easily still associate information with its metadata.

note that I'm using the `__post_init__()` dunder method to do some data validation - although an alterantive solution is using a few tricks around the `dataclasses.field` object instead.

In [ ]:
def get_level_of_python_descriptor(level_of_python: int):
    """
    Returns a string descriptor for the given level of Python proficiency.

    Parameters:
    level_of_python (int): An integer representing the level of Python proficiency.
                           Accepted values are:
                           0 - No Python
                           1 - Basic Python
                           2 - Intermediate Python
                           3 - Advanced Python

    Returns:
    str: A string describing the level of Python proficiency.

    Raises:
    ValueError: If the input value is not in the range 0-3.
    """
    match level_of_python:
        case 0:
            return "No Python"
        case 1:
            return "Basic Python"
        case 2:
            return "Intermediate Python"
        case 3:
            return "Advanced Python"
        case _:
            raise ValueError("Only values of 0-3 accepted")
        
def validate_annotations(tested_instance):
    """
    Validates that the attributes of the given instance match their type annotations.

    Args:
        tested_instance: The instance whose attributes are to be validated.

    Raises:
        AssertionError: If any attribute does not match its type annotation.
    """
    type_hints = get_type_hints(tested_instance)
    for k, v in type_hints.items():
        attr_value = getattr(tested_instance, k)
        if v == List[str]:
            assert isinstance(attr_value, list) and all(isinstance(i, str) for i in attr_value), (
                f"Expected {k} to be of type List[str], got {type(attr_value)} instead."
            )
        else:
            assert isinstance(attr_value, v), f"Expected {k} to be of type {v}, got {type(attr_value)} instead."


@dataclass(frozen=True, slots=True)
class Project:
    """
    A class to represent a project with specific attributes and requirements.
    Attributes:
    ----------
    name : str
        The name of the project.
    hours_saved_per_month : int
        The number of hours saved per month by implementing the project.
    hours_to_implement : int
        The number of hours required to implement the project.
    skills_required : List[str]
        A list of skills required to implement the project. Valid skills are "APIs", "Excel", and "SQL".
    python_level_required : int
        The level of Python proficiency required for the project. 
        Valid levels are:
        0 - No Python required
        1 - Basic
        2 - Intermediate
        3 - Advanced
    Methods:
    -------
    __post_init__():
        Validates the annotations and checks the validity of the python_level_required and skills_required attributes.
    __repr__():
        Returns a string representation of the project, including its name, hours saved per month, hours to implement, 
        required skills, and the level of Python proficiency required.
    """
    
    name: str
    hours_saved_per_month: int
    hours_to_implement: int
    skills_required: List[str]
    python_level_required: int  # 1 for basic, 2 for intermediate, 3 for advanced
    
    def __post_init__(self):
        """
        Post-initialization method to validate the annotations and ensure that the 
        attributes `python_level_required` and `skills_required` meet the specified criteria.
        Raises:
            AssertionError: If `python_level_required` is not in {0, 1, 2, 3}.
            AssertionError: If `skills_required` contains elements not in {"APIs", "Excel", "SQL"}.
        """
        validate_annotations(self)
        
        assert self.python_level_required in {0, 1, 2, 3}, "Invalid Python level required"
        assert set(self.skills_required).issubset({"APIs", "Excel", "SQL"}), "Invalid skills required"
        
    def __repr__(self):
        """
        Returns a string representation of the object, detailing the name, hours saved per month,
        hours required to implement, required skills, and the level of Python skills required.

        Returns:
            str: A formatted string representation of the object.
        """
        representation = f"""{self.name}: {self.hours_saved_per_month} hours per month to be saved against {self.hours_to_implement} hours to implement.
        Requires {", ".join(self.skills_required)} and {get_level_of_python_descriptor(self.python_level_required)} skills"""
        return representation
    
@dataclass(frozen=True, slots=True)
class TeamMember:
    """
    A class to represent a team member with specific attributes and skills.
    Attributes:
    ----------
    name : str
        The name of the team member.
    hours_available : int
        The number of hours the team member is available for projects in the financial year.
    skills : List[str]
        A list of skills the team member possesses. Valid skills are "APIs", "Excel", and "SQL".
    python_level : int
        The proficiency level of the team member in Python. Valid levels are 0, 1, 2, and 3.
    Methods:
    -------
    __post_init__():
        Validates the annotations and ensures the python_level and skills are within the allowed values.
    __repr__():
        Returns a string representation of the team member's availability, skills, and Python proficiency level.
    """
    name: str
    hours_available: int
    skills: List[str]
    python_level: int
    
    
    def __post_init__(self):
        """
        Post-initialization method for validating the attributes of the instance.
        This method performs the following validations:
        - Ensures that the `python_level` attribute is one of the valid levels: 0, 1, 2, or 3.
        - Ensures that the `skills` attribute is a subset of the allowed skills: {"APIs", "Excel", "SQL"}.
        Raises:
            AssertionError: If `python_level` is not in {0, 1, 2, 3}.
            AssertionError: If `skills` contains any invalid skill.
        """
        validate_annotations(self)
        
        assert self.python_level in {0, 1, 2, 3}, "Invalid Python level"
        assert set(self.skills).issubset({"APIs", "Excel", "SQL"}), "Invalid skills"
    
    def __repr__(self):
        """
        Return a string representation of the object.
        The representation includes the name of the person, the number of hours 
        they have available for projects in the current financial year, their 
        skills, and their level of proficiency in Python.
        Returns:
            str: A formatted string describing the person's availability, skills, 
            and Python proficiency level.
        """
        representation = f"""{self.name} has {self.hours_available} hours available this financial year for projects.
            {self.name} has {", ".join(self.skills)} skills, and abilities up to {get_level_of_python_descriptor(self.python_level)}"""
        return representation

In [ ]:
# creating our projects
projects = [
    Project("Secondary Deliverables reporting", 20, 160, ["SQL", "Excel"], 2),
    Project("I&M Work Order generation", 40, 200, ["SQL", "Excel", "APIs"], 2),
    Project("ISS", 5, 80, ["SQL"], 2),
    Project("Network Interventions lists", 5, 40, ["SQL"], 1),
    Project("Pulse Report report generation", 10, 80, ["SQL", "APIs"], 3),
]
# creating our team members

team = [
    TeamMember("Kermit the Frog", 80, ["SQL", "Excel"], 2),
    TeamMember("Sweetums", 40, ["Excel"], 1),
    TeamMember("Dr Bunsen Honeydew", 60, ["APIs", "SQL", "Excel"], 3),
    TeamMember("Fozzie the Bear", 120, ["SQL", "Excel"], 2),
    TeamMember("Pepe the King Prawn", 20, ["SQL", "Excel"], 1),
    TeamMember("Rowlf the Dog", 40, ["APIs", "SQL", "Excel"], 3),
]

In [7]:
projects

[Secondary Deliverables reporting: 20 hours per month to be saved against 160 hours to implement.
         Requires SQL, Excel and Intermediate Python skills,
 I&M Work Order generation: 40 hours per month to be saved against 200 hours to implement.
         Requires SQL, Excel, APIs and Intermediate Python skills,
 ISS: 5 hours per month to be saved against 80 hours to implement.
         Requires SQL and Intermediate Python skills,
 Network Interventions lists: 5 hours per month to be saved against 40 hours to implement.
         Requires SQL and Basic Python skills,
 Pulse Report report generation: 10 hours per month to be saved against 80 hours to implement.
         Requires SQL, APIs and Advanced Python skills]

In [8]:
team

[Kermit the Frog has 80 hours available this financial year for projects.
             Kermit the Frog has SQL, Excel skills, and abilities up to Intermediate Python,
 Sweetums has 40 hours available this financial year for projects.
             Sweetums has Excel skills, and abilities up to Basic Python,
 Dr Bunsen Honeydew has 60 hours available this financial year for projects.
             Dr Bunsen Honeydew has APIs, SQL, Excel skills, and abilities up to Advanced Python,
 Fozzie the Bear has 120 hours available this financial year for projects.
             Fozzie the Bear has SQL, Excel skills, and abilities up to Intermediate Python,
 Pepe the King Prawn has 20 hours available this financial year for projects.
             Pepe the King Prawn has SQL, Excel skills, and abilities up to Basic Python,
 Rowlf the Dog has 40 hours available this financial year for projects.
             Rowlf the Dog has APIs, SQL, Excel skills, and abilities up to Advanced Python]

Each of our types of class are working wonderfully, and have helpful representations. Now we can get started with the optimiser.

## Formulating the optimiser

Now that we have our data ready to go, we need to get out optimiser set up.

These are our general parameters:
- **Goal:** maximise the total hours saved per month by selecting the best combination of projects, and assigning the approparite team members to them.
- **Constraints:**
  - A maximum of **two** projects can run simultaneously,
  - Team members assigned to a project must have the appropriate skills,
  - Team members cannot be assigned more horus than they have available,
  - The python proficiency of a team member must meet or exceed the complexity of the project.

Lets use `PuLP` to get the problem set up as a model.

In [ ]:
problem = LpProblem(name="Project-Selection", sense=LpMaximize)

project_variables = LpVariable.dicts(
    name="SelectProject", indices=[p.name for p in projects], cat="Binary"
)

team_project_variables = LpVariable.dicts(
    name="Assign",
    indices=[(tm.name, p.name) for tm in team for p in projects],
    cat="Binary",
)

We've defined the decision variables now. They're `Binary` - `1` for if the project or team member is picked by the optimiser, or `0` if it isn't. We'll load them in to the "problem" next


In [ ]:
problem += (
    lpSum([project_variables[p.name] * p.hours_saved_per_month for p in projects]),
    "Total Hours Saved",
)

`project_variables[p.name] * p.hours_saved_per_month` looks a little odd, given the fact that we're seemingly circulating a name with hours saved - but let's take that operation on its own and it should become more apparant the way that `PuLP` works

In [ ]:
project_variables["ISS"] * 4

4*SelectProject_ISS + 0

That output looks a bit like $4x + 0$, if we take SelectProject_ISS to be $x$. We're creating a structure that is parallel to a mathematical expression, just like the ones we were working with in the crash course...

`problem` is now a good representation of our *objective function*:

$$ Z = 20x_1 + 40x_2 + 5x_3 + 5x_4 + 10x_5 $$

### Constraints

Let's add the constraints, one by one.

#### Contraint 1: Maximum of two projects running simultaneously

Let's write in the constraint
$$ x_1 + x_2 + x_3 + x_4 + x_5 \leq 2 $$

or otherwise:
$$\sum_ix_i \leq 2$$

In [ ]:
# Maximum of two projects running simultaneously
problem += (
    lpSum([project_variables[p.name] for p in projects]) <= 2,
    "Max_Concurrent_Projects",
)

#### Constraint 2: Team members are only assigned if they have the relevant skills
 `(tm.name, p.name)` is a tuple representing the combination of team member and project name. As these are `Binary`, if the team member is on a project, the value would be `1`. 

In [ ]:
# Team members can only be assigned to selected projects
for tm in team:
    for p in projects:
        problem += (
            team_project_variables[(tm.name, p.name)] <= project_variables[p.name],
            f"Assign_{tm.name}_{p.name}_OnlyIfProjectSelected",
        )

#### Constraint 3: Team members can only work on a project if they meet the skills and Python level


In [33]:
# Ensure team members meet skill and Python level requirements
for p in projects:
    for tm in team:
        if (
            not set(p.skills_required).issubset(set(tm.skills))
            or tm.python_level < p.python_level_required
        ):
            problem += (
                team_project_variables[(tm.name, p.name)] == 0,
                f"Skill_Python_Level_Match_{tm.name}_{p.name}",
            )

In [34]:
# Team member hours availability
for tm in team:
    total_hours_assigned = lpSum(
        [team_project_variables[(tm.name, p.name)] * p.hours_to_implement for p in projects]
    )
    problem += total_hours_assigned <= tm.hours_available, f"Hours_Available_{tm.name}"

In [35]:
# Each selected project must have at least one team member assigned
for p in projects:
    total_team_assigned = lpSum(
        [team_project_variables[(tm.name, p.name)] for tm in team]
    )
    problem += total_team_assigned >= project_variables[p.name], f"Team_Assignment_{p.name}"